In [ ]:
!pip3 install category_encoders
!pip3 install catboost
!pip3 install xgboost

In [3]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import scipy , math
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
from tqdm import tqdm
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_fscore_support, roc_curve, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
import category_encoders as ce
import random
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp -r '/content/drive/MyDrive/fashion_data/jobthon' jobthon
!cp jobthon/train_Df64byy.csv train.csv
!cp jobthon/test_YCcRUnU.csv test.csv
!cp jobthon/sample_submission_QrCyCoT.csv sample_submission.csv

In [326]:
df = pd.read_csv('train.csv')
df.drop(['ID'], inplace = True, axis = 1)
df.head(5)

,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0


##1)Couple_Response_Rate

In [337]:
#In case of Joint account if they are couples what is the chance of getting Response as 1

couple_count = len(df[(df.Reco_Insurance_Type == 'Joint') & (df.Is_Spouse == 'Yes')])
num = len(df[(df.Reco_Insurance_Type == 'Joint') & (df.Is_Spouse == 'Yes') & (df.Response == 1)])

print(num, couple_count)    ### At test time just use num and couple_count dont calculate it
df['couple_response_rate'] = df.apply(lambda x : (num/couple_count) if (x['Reco_Insurance_Type'] == 'Joint' and x['Is_Spouse'] == 'Yes') else 0, axis = 1)

2052 8422


#Missing value of Holding Policy Type 
   Here we are considering that if policy type is missing then that customer do not have policy so, create new policy type indicating customer do not have any policy. Same is true for 'Holding_Policy_Duration'


In [328]:
df['Hold_Policy'] = df['Holding_Policy_Type'].apply(lambda x : 0 if math.isnan(x) else 1)
df['Holding_Policy_Type'] = df['Holding_Policy_Type'].apply(lambda x : 0.0 if math.isnan(x) else x)

df['Holding_Policy_Duration'] = df['Holding_Policy_Duration'].apply(lambda x : 15.0 if x == '14+' else x)
df['Holding_Policy_Duration'] = df['Holding_Policy_Duration'].apply(lambda x : 0 if math.isnan(float(x)) else x)
df['Holding_Policy_Duration'] = df['Holding_Policy_Duration'].apply(lambda x : int(float(x))).astype(int)

df['Health_Missing'] = df['Health Indicator'].apply(lambda x : 1 if str(x)[0] != 'X' else 0)
df['Reco_Policy_Premium'] = df['Reco_Policy_Premium'].apply(lambda x : int(float(x))).astype(int)

df.fillna(value = 'X0', inplace = True)

cat4 = ['City_Code', 'Region_Code', 'Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse', \
           'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat', 'Hold_Policy', 'Health_Missing']

numeric_features = ['Upper_Age', 'Lower_Age', 'Reco_Policy_Premium', 'couple_response_rate']

# All categorical features got converted into labels using label encoder Nan is not touched

label_dict = {}
def label(df):
    _df = df.copy()
    for col in cat4 :
        le = LabelEncoder()
        idx = ~_df[col].isna()
        _df.loc[idx, col] = le.fit_transform(_df.loc[idx, col])
        label_dict[col] = le
    return _df

df_new = label(df)

#Adding New Features

##2) Is Big City

In [258]:
group_city = df_new.groupby('City_Code')['Region_Code'].agg('nunique')
df_new['is_big_city'] = df_new['City_Code'].apply(lambda x : group_city.at[x])
numeric_features.append('is_big_city')

##3) Prob_Estimate

In [259]:
# Old Guy + Already hold policy for long duration (Old Customer) + Rich meaning owns the house

df_prob = pd.DataFrame()
df_prob['old_guy'] = df_new.Upper_Age.apply(lambda x : 1 if x > 40 else 0)
df_prob['old_cust'] =  df_new.Holding_Policy_Duration.apply(lambda x : 1 if x > 7 else 0)
df_prob['is_rich'] = df_new.Accomodation_Type.apply(lambda x : 1 if x == 'Owned' else 0)
df_new['prob_estimate'] = (df_prob['old_guy'] + df_prob['old_cust'] + df_prob['is_rich']) / 3
numeric_features.append('prob_estimate')

##4)City_Response

In [260]:
city_response = {}

for city in set(df_new.City_Code) :
    city_response[city] = len(df_new[(df_new.City_Code == city) & (df_new.Response == 1)]) / len(df_new[df_new.Response == 1])

df_new['city_response_rate'] = df_new.City_Code.apply(lambda x : city_response[x])
numeric_features.append('city_response_rate')
data = df_new.copy()

In [261]:
cat4.remove('Region_Code')   ## must run if features are not removed using above stats techniques / tests

#2)XGBoost

In [262]:
df_new2 = data.copy()
new_df = df_new2.copy()
Y = new_df.Response
new_df.drop(['Response'], axis = 1, inplace = True)
new_df = pd.get_dummies(new_df, columns = cat4)
x_train, x_test, y_train, y_test = train_test_split(new_df, Y, stratify = Y, test_size = 0.2, random_state = 24)

#Applying Frequency Encoding on Region Code

In [263]:
## Dont run this cell if you have used stats methods to the remove features

fq = x_train.groupby('Region_Code').size()/len(x_train)  

x_train['Freq_Region'] = x_train['Region_Code'].apply(lambda x : fq.at[x])
x_train.drop(['Region_Code'], axis = 1, inplace = True)

x_test['Freq_Region'] = x_test['Region_Code'].apply(lambda x : fq.at[x] if x in fq.index else 0.0)
x_test.drop(['Region_Code'], axis = 1, inplace = True)
numeric_features.append('Freq_Region')

#Tune The Model

In [264]:
scaler = StandardScaler()
x_train_fitted = scaler.fit_transform(x_train)
x_test_fitted = scaler.transform(x_test)

#x_train_fitted, x_test_fitted = x_train, x_test     ## uncomment it when you do not use scaler

x_train_new, y_train_new = x_train_fitted, y_train   ## Added this line in case you do not want to try any class balancing technique

oversample = RandomOverSampler(random_state = 12)
x_train_new, y_train_new = oversample.fit_resample(x_train_fitted, y_train)

#undersample = RandomUnderSampler(random_state = 12)
#x_train_new, y_train_new = undersample.fit_resample(x_train_fitted, y_train)

#smote = SMOTE(random_state = 50, k_neighbors = 5)
#x_train_new, y_train_new = smote.fit_resample(x_train_fitted, y_train)


In [ ]:
for lr in np.linspace(0.01, 0.1, 10) :
    for est in list(range(40, 150, 20)) :
        for depth in [2, 5, 7,8, 9, 10] :
            model= XGBClassifier(silent=False, 
                      learning_rate=lr,  
                      colsample_bytree = 0.4,
                      subsample = 0.6,
                      n_estimators=est, 
                      max_depth=depth)
            model.fit(x_train_new, y_train_new)
            print(lr, est, depth, 'Train = ', roc_auc_score(y_train_new, model.predict_proba(x_train_new)[:, 1]))
            print('Validation = ', roc_auc_score(y_test, model.predict_proba(x_test_fitted)[:, 1]))


In [265]:
model= XGBClassifier(silent=False, 
                      learning_rate= 0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.6,
                      n_estimators=140, 
                      max_depth=10)
model.fit(x_train_new, y_train_new)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=140, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=0.6, verbosity=1)

#Submitting File

In [375]:
test_df = pd.read_csv('test.csv')
ids = test_df.ID.values
test_df.drop(['ID'], inplace = True, axis = 1)
test_df.head()

,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0
1,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8
2,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0
3,C3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0
4,C1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0


In [376]:
test_df['couple_response_rate'] = test_df.apply(lambda x : (num/couple_count) if (x['Reco_Insurance_Type'] == 'Joint' and x['Is_Spouse'] == 'Yes') else 0, axis = 1)

In [377]:
def label_test(df) :
    _df = df.copy()
    for col in cat4:
        le = label_dict[col]
        dic = dict(zip(le.classes_, le.transform(le.classes_)))
        _df[col] = _df[col].map(dic).fillna(0).astype(int) 
    return _df

In [378]:
test_df['Hold_Policy'] = test_df['Holding_Policy_Type'].apply(lambda x : 0 if math.isnan(x) else 1)
test_df['Holding_Policy_Type'] = test_df['Holding_Policy_Type'].apply(lambda x : 0.0 if math.isnan(x) else x)

test_df['Holding_Policy_Duration'] = test_df['Holding_Policy_Duration'].apply(lambda x : 15.0 if x == '14+' else x)
test_df['Holding_Policy_Duration'] = test_df['Holding_Policy_Duration'].apply(lambda x : 0 if math.isnan(float(x)) else x)
test_df['Holding_Policy_Duration'] = test_df['Holding_Policy_Duration'].apply(lambda x : int(float(x))).astype(int)

test_df['Health_Missing'] = test_df['Health Indicator'].apply(lambda x : 1 if str(x)[0] != 'X' else 0)
test_df['Reco_Policy_Premium'] = test_df['Reco_Policy_Premium'].apply(lambda x : int(float(x))).astype(int)

test_df.fillna(value = 'X0', inplace = True)

cat4 = ['City_Code', 'Region_Code', 'Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse', \
           'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat', 'Hold_Policy', 'Health_Missing']

numeric_features = ['Upper_Age', 'Lower_Age', 'Reco_Policy_Premium', 'couple_response_rate']

test_df_new = label_test(test_df)

In [379]:
test_df_new['is_big_city'] = test_df_new['City_Code'].apply(lambda x : group_city.at[x] if x in group_city.index else 0)
numeric_features.append('is_big_city')

In [380]:
# Old Guy + Already hold policy for long duration (Old Customer) + Rich meaning owns the house

df_prob = pd.DataFrame()
df_prob['old_guy'] = test_df_new.Upper_Age.apply(lambda x : 1 if x > 40 else 0)
df_prob['old_cust'] =  test_df_new.Holding_Policy_Duration.apply(lambda x : 1 if x > 7 else 0)
df_prob['is_rich'] = test_df_new.Accomodation_Type.apply(lambda x : 1 if x == 'Owned' else 0)
test_df_new['prob_estimate'] = (df_prob['old_guy'] + df_prob['old_cust'] + df_prob['is_rich']) / 3
numeric_features.append('prob_estimate')

In [381]:
test_df_new['city_response_rate'] = test_df_new.City_Code.apply(lambda x : city_response[x] if x in city_response else 0)
numeric_features.append('city_response_rate')

In [382]:
cat4.remove('Region_Code')   ## must run if features are not removed using above stats techniques / tests

In [383]:
test_df_new = pd.get_dummies(test_df_new, columns = cat4)
test_df_new = test_df_new.reindex(columns=new_df.columns, fill_value=0)
test_df_new.head()

,Region_Code,Upper_Age,Lower_Age,Reco_Policy_Premium,couple_response_rate,is_big_city,prob_estimate,city_response_rate,City_Code_0,City_Code_1,City_Code_2,City_Code_3,City_Code_4,City_Code_5,City_Code_6,City_Code_7,City_Code_8,City_Code_9,City_Code_10,City_Code_11,City_Code_12,City_Code_13,City_Code_14,City_Code_15,City_Code_16,City_Code_17,City_Code_18,City_Code_19,City_Code_20,City_Code_21,City_Code_22,City_Code_23,City_Code_24,City_Code_25,City_Code_26,City_Code_27,City_Code_28,City_Code_29,City_Code_30,City_Code_31,...,Holding_Policy_Duration_7,Holding_Policy_Duration_8,Holding_Policy_Duration_9,Holding_Policy_Duration_10,Holding_Policy_Duration_11,Holding_Policy_Duration_12,Holding_Policy_Duration_13,Holding_Policy_Duration_14,Holding_Policy_Duration_15,Holding_Policy_Type_0,Holding_Policy_Type_1,Holding_Policy_Type_2,Holding_Policy_Type_3,Holding_Policy_Type_4,Reco_Policy_Cat_0,Reco_Policy_Cat_1,Reco_Policy_Cat_2,Reco_Policy_Cat_3,Reco_Policy_Cat_4,Reco_Policy_Cat_5,Reco_Policy_Cat_6,Reco_Policy_Cat_7,Reco_Policy_Cat_8,Reco_Policy_Cat_9,Reco_Policy_Cat_10,Reco_Policy_Cat_11,Reco_Policy_Cat_12,Reco_Policy_Cat_13,Reco_Policy_Cat_14,Reco_Policy_Cat_15,Reco_Policy_Cat_16,Reco_Policy_Cat_17,Reco_Policy_Cat_18,Reco_Policy_Cat_19,Reco_Policy_Cat_20,Reco_Policy_Cat_21,Hold_Policy_0,Hold_Policy_1,Health_Missing_0,Health_Missing_1
0,126,30,30,11934,0.000000,741,0.000000,0.180850,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,5,69,68,32204,0.243648,328,0.333333,0.072815,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
2,457,28,28,9240,0.000000,741,0.000000,0.180850,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
3,985,23,23,9086,0.000000,441,0.000000,0.095094,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
4,781,75,75,22534,0.000000,741,0.333333,0.180850,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [384]:
test_df_new['Freq_Region'] = test_df_new['Region_Code'].apply(lambda x : fq.at[x] if x in fq.index else 0.0)
test_df_new.drop(['Region_Code'], axis = 1, inplace = True)
numeric_features.append('Freq_Region')

In [385]:
sub_df = scaler.transform(test_df_new)

In [386]:
predicted_prob = model.predict_proba(sub_df)[:, 1]

In [387]:
submission_xg = pd.DataFrame({'ID' : ids , 'Response' : predicted_prob})
submission_xg.to_csv('submission_xg.csv', index= False)
print(len(predicted_prob), len(sub_df))

21805 21805


In [388]:
xg_csv = pd.read_csv('submission_xg.csv')
xg_csv.head()

,ID,Response
0,50883,0.515321
1,50884,0.477294
2,50885,0.533003
3,50886,0.461483
4,50887,0.485386
